# Předzpracování datasetu CIFAR100

Tento notebook slouží k předzpracování datasetu CIFAR100. Pro dataset jsou vytvořeny augmentované záznamy a předpočítány logity. Nejprve jsou načteny všechny potřebné knihovny včetně vlastní sbírky objektů a funkcí.

In [1]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from torch.utils.data import DataLoader, Subset, ConcatDataset
from transformers import AutoModelForImageClassification
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import os 

import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Ověření, že GPU je k dispozici a balíček torch je správně nakonfigurován.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [3]:
base.reset_seed(42)

Získání základních transformací pro spuštění inference nad učitelským modelem. Základní transformace primárně obrázky zvětšují na rozměr 224x224 a normalizují barevné kanály. 

Transformace sloužící k augmentaci poté tyto kroky rozšiřují o rotaci a převracení os, vždy s určitou pravděpodobností. 

Transformace pochází z balíčku torchvision.

Následně dochází k načtení datasetu.

In [4]:
transform = base.base_transforms()
augment_transform = base.aug_transforms()

train = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device=device)
train_aug = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=augment_transform, device=device)


test = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=False, transform=transform, device=device)
eval = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device=device)

Provedení stratifikovaného rozdělení pro získání validační části datasetu.

In [5]:
train_idx, validation_idx = train_test_split(np.arange(len(train)),
                                             test_size=0.2,
                                             random_state=42,
                                             shuffle=True,
                                             stratify=train.labels)

In [6]:
train = Subset(train, train_idx)
train_aug = Subset(train_aug, train_idx)
eval = Subset(eval, validation_idx)

Vytvoření dataloaderů a získání předtrénovaného učitele z HuggingFace.

In [7]:
train_dataloader = DataLoader(train, batch_size=128, shuffle=False)
train_dataloader_aug = DataLoader(train_aug, batch_size=128, shuffle=False)

In [8]:
eval_dataloder = DataLoader(eval, batch_size=128, shuffle=False)
test_dataloader = DataLoader(test, batch_size=128, shuffle=False)

In [9]:
model = AutoModelForImageClassification.from_pretrained(
    "Ahmed9275/Vit-Cifar100",
    num_labels=100,
)

model.to(device)
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/cifar100/teacher.pth")

In [10]:
model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

Vypočtení logitů nad dílčími částmi datasetu. CIFAR100 nemá trénovací část rozdělenou do více souborů, všechna data jsou v rámci jednoho, není tedy třeba cyklu.

In [11]:
logits = base.generate_logits(train_dataloader, model)
logits_aug = base.generate_logits(train_dataloader_aug, model)

logits_eval = base.generate_logits(eval_dataloder, model)
logits_test = base.generate_logits(test_dataloader, model)

Generating logits for given dataset:   0%|          | 0/313 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/313 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/79 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/79 [00:00<?, ?it/s]

Načtení podkladových souborů a oddělení validační části od trénovací. Odděleny jsou stejné záznamy jako v případě stratifikovaného rozdělení přímo nad datasetem objektu. Soubory tedy odpovídají jednotlivým objektům.

Následně jsou do jednotlivých souborů uloženy spočtené logity a soubory jsu uloženy.

In [12]:
data_file = base.unpickle(f"{os.path.expanduser('~')}/data/100/cifar-100-python/train")
testing = base.unpickle(f"{os.path.expanduser('~')}/data/100/cifar-100-python/test")

In [13]:
data = {key: [value[i] for i in train_idx] for key, value in data_file.items() if key != b"batch_label"}    
eval_data = {key: [value[i] for i in validation_idx] for key, value in data_file.items() if key != b"batch_label"}    

In [14]:
data[b"logits"] = logits
data[b"logits_aug"] = logits_aug

eval_data[b"logits"] = logits_eval
testing[b"logits"] = logits_test

In [15]:
base.pickle_up(f"{os.path.expanduser('~')}/data/100-logits/cifar-100-python/test", testing)
base.pickle_up(f"{os.path.expanduser('~')}/data/100-logits/cifar-100-python/train", data)
base.pickle_up(f"{os.path.expanduser('~')}/data/100-logits/cifar-100-python/eval", eval_data)

In [13]:
dataset_part = base.get_dataset_part()

Pro kontrolu jsou soubory načteny a je spočtena správnost predikcí učitele.

In [17]:
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform)
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform)

print(base.check_acc(test, "Accuracy for test dataset is:"))
print(base.check_acc(eval, "Accuracy for eval dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy for test dataset is: 0.8182


Calculating accuracy based on the saved logits:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy for eval dataset is: 0.9445


V případě augmentované trénovací části opět narážíme na nízkou správnost.

In [18]:
train_aug = base.CustomCIFAR100L(f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)
train = base.CustomCIFAR100L(f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform)
train_combo = ConcatDataset([train, train_aug])

In [19]:
print(base.check_acc(train, "Accuracy for train dataset is:"))
print(base.check_acc(train_aug, "Accuracy for augmeted train dataset is:"))
print(base.check_acc(train_combo, "Accuracy for combined dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/40000 [00:00<?, ?it/s]

Accuracy for train dataset is: 0.94035


Calculating accuracy based on the saved logits:   0%|          | 0/40000 [00:00<?, ?it/s]

Accuracy for augmeted train dataset is: 0.6315


Calculating accuracy based on the saved logits:   0%|          | 0/80000 [00:00<?, ?it/s]

Accuracy for combined dataset is: 0.785925


Provedení filtrace augmentovaného datasetu dle popsaného mechanismu.

In [20]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

In [ ]:
print(len(train_aug))

25912


In [22]:
print(base.check_acc(train_aug, "Accuracy for filtered augmented dataset is:"))
print(base.check_acc(train_combo, "Accuracy for combined dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/25912 [00:00<?, ?it/s]

Accuracy for filtered augmented dataset is: 0.9609833281877123


Calculating accuracy based on the saved logits:   0%|          | 0/65912 [00:00<?, ?it/s]

Accuracy for combined dataset is: 0.9484615851438282


Výsledné statistiky učitele.

In [23]:
base.count_parameters(model)

model size: 327.589MB.
Total Trainable Params: 85875556.


,Modules,Parameters
0,vit.embeddings.cls_token,768
1,vit.embeddings.position_embeddings,151296
2,vit.embeddings.patch_embeddings.projection.weight,589824
3,vit.embeddings.patch_embeddings.projection.bias,768
4,vit.encoder.layer.0.attention.attention.query....,589824
...,...,...
195,vit.encoder.layer.11.layernorm_after.bias,768
196,vit.layernorm.weight,768
197,vit.layernorm.bias,768
198,classifier.weight,76800


In [24]:
train_part_cpu = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device="cpu")
cpu_data_loader = DataLoader(train_part_cpu, batch_size=1, shuffle=False)
cpu_benchmark = base.BenchMarkRunner(model, cpu_data_loader, "cpu", 1000)

print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  164.83 ms
  1 measurement, 1000 runs , 4 threads


In [25]:
train_part_gpu = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device="cuda")
gpu_data_loader = DataLoader(train_part_gpu, batch_size=1, shuffle=False)
gpu_benchmark = base.BenchMarkRunner(model, gpu_data_loader, "cuda", 1000)


print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  7.33 ms
  1 measurement, 1000 runs , 4 threads


In [15]:
test_data_preds = []
test_data_labels = []

for index, val in enumerate(test):
    test_data_preds.append(torch.topk(val["logits"], k=1).indices.numpy()[0])
    test_data_labels.append(val["labels"])

In [17]:
f1 = f1_score(test_data_labels, test_data_preds, average="macro")
acc = accuracy_score(test_data_labels, test_data_preds)
precision = precision_score(test_data_labels, test_data_preds, average="macro")
recall = recall_score(test_data_labels, test_data_preds, average="macro")

print(f"F1 score: {f1}")
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

F1 score: 0.820402306187161
Accuracy: 0.8182
Precision: 0.8323971511869911
Recall: 0.8182
